Import thư viện cần thiết


In [16]:
import pandas as pd
import re
from langdetect import detect, LangDetectException
import ast
import csv
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import quote
import json
import pandas as pd
import re
import logging
import requests
import time
from datetime import datetime
import google.generativeai as genai
from tqdm import tqdm
import os


- Tạo lại cấu trúc chuẩn cho 2 file
    - File Nhacvn: path: "normalize_Nhacvn_output.csv" từ {urls,title,artist,composer,genre,lyrics} ==> {title ,composers, lyricists, year, genres, lyrics, urls, source, note} , cột không có thì để là Null
    - File tkaraoke : path:"normalize_tkaraoke_output.csv" từ {urls,title,artist,lyrics} ==> {title ,composers, lyricists, year, genres, lyrics, urls, source, note} , cột không có thì để trống hoặc null tùy theo kiểu dữ liệu




# NHACVN chuan hoa cấu trúc

In [2]:
# === Đọc file gốc ===
df = pd.read_csv("normalize_Nhacvn_output.csv")

# === Hàm tiện ích để chuẩn hóa dạng list ===
def ensure_list(x):
    if pd.isna(x) or x == "Null":
        return []
    if isinstance(x, list):
        return [str(v).strip() for v in x if str(v).strip()]
    if isinstance(x, str):
        try:
            val = ast.literal_eval(x)
            if isinstance(val, list):
                return [str(v).strip() for v in val if str(v).strip()]
        except:
            return [v.strip() for v in x.split(",") if v.strip()]
    return [str(x).strip()]

# === Tạo dataframe mới theo schema chuẩn ===
df_new = pd.DataFrame({
    "title": df.get("title", pd.Series([""] * len(df))),
    "composers": df.get("composer", pd.Series([""] * len(df))).apply(ensure_list),
    "lyricists": pd.Series([[]] * len(df)),     # không có trong Nhacvn
    "year": pd.Series([None] * len(df)),        # chưa có dữ liệu năm
    "genres": df.get("genre", pd.Series([""] * len(df))).apply(ensure_list),
    "lyrics": df.get("lyrics", pd.Series([""] * len(df))),
    "urls": df.get("urls", pd.Series([""] * len(df))).apply(ensure_list),
    "source": pd.Series([""] * len(df)),        # trống theo kiểu text
    "note": pd.Series([""] * len(df))           # trống theo kiểu text
})

# === Xuất ra CSV & JSON ===
df_new.to_csv("normalized_Nhacvn_standard.csv", index=False, encoding="utf-8-sig")



# CHUAN HOA CAU TRUC tkaraoke

In [ ]:
import pandas as pd
import re
import ast

# === Đọc file gốc ===
df = pd.read_csv("normalize_tkaraoke_output.csv")

def ensure_list(x):
    if pd.isna(x) or x == "Null":
        return []
    if isinstance(x, list):
        return [str(v).strip() for v in x if str(v).strip()]
    if isinstance(x, str):
        # nếu là chuỗi chứa list literal, thử parse
        try:
            import ast
            val = ast.literal_eval(x)
            if isinstance(val, list):
                return [str(v).strip() for v in val if str(v).strip()]
        except:
            return [v.strip() for v in x.split(",") if v.strip()]
    return [str(x).strip()]

def extract_authors(text):
    """
    Trả về (composers:list[str], lyricists:list[str])
    - Xử lý các dạng: "Tác giả:NhạcDBaola&JustaTee, lờiJustaTee"
    - Hoạt động với "Nhạc", "Lời", "Thơ" viết dính hoặc cách
    - Tách theo ',', '&', hoặc từ 'và' (word boundary)
    - Giữ unique trong từng danh sách nhưng KHÔNG loại tên giữa 2 role
    """
    composers, lyricists = [], []
    if not isinstance(text, str) or not text.strip():
        return [], []

    # 1) loại prefix
    t = text.replace("Tác giả:", "").strip()

    # 2) xóa ký tự ẩn nếu có
    t = re.sub(r'[\u200B-\u200D\uFEFF\u00A0]', '', t)

    # 3) thêm dấu cách khi 'Nhạc'/'Lời'/'Thơ' dính liền với tên (ví dụ "NhạcDBaola")
    t = re.sub(r'(?i)(Nhạc|Lời|Thơ)(?=[^\s:])', r'\1 ', t)

    # 4) chuẩn hóa khoảng trắng
    t = re.sub(r'\s+', ' ', t).strip()

    # 5) tìm tất cả khối "Nhạc ...", "Lời ...", "Thơ ..." (mỗi khối kéo tới trước keyword tiếp theo hoặc tới EOL)
    pattern = re.compile(r'(?i)(Nhạc|Lời|Thơ)\s*:?\s*(.*?)(?=(?:Nhạc|Lời|Thơ|$))', flags=re.IGNORECASE)
    matches = list(pattern.finditer(t))

    split_re = re.compile(r'\s*(?:,|&|\bvà\b)\s*', flags=re.IGNORECASE)

    if matches:
        for m in matches:
            role = m.group(1).strip().lower()
            names_raw = m.group(2).strip()
            # tách theo , & và từ 'và' (word-boundary)
            names = [n.strip() for n in split_re.split(names_raw) if n.strip()]
            # chuẩn hóa tên (loại khoảng trắng thừa)
            names = [n for n in names if n]
            # title-case tên (tuỳ bạn, có thể bỏ .title() nếu muốn giữ nguyên)
            names = [n.strip() for n in names]
            if role.startswith('nhạc'):
                composers.extend(names)
            else:  # 'lời' hoặc 'thơ' -> lyricists
                lyricists.extend(names)
    else:
        # không thấy keyword -> coi toàn bộ làm composers, tách theo separators
        names = [n.strip() for n in split_re.split(t) if n.strip()]
        composers.extend(names)

    # unique từng list, giữ thứ tự xuất hiện
    def unique_preserve_order(seq):
        seen = set()
        out = []
        for x in seq:
            key = x.strip()
            if key and key not in seen:
                seen.add(key)
                out.append(key)
        return out

    composers = unique_preserve_order(composers)
    lyricists = unique_preserve_order(lyricists)

    return composers, lyricists


# === Tạo cấu trúc dữ liệu chuẩn ===
records = []
for _, row in df.iterrows():
    composers, lyricists = extract_authors(row.get("artist", ""))
    urls = ensure_list(row.get("urls", ""))

    records.append({
        "title": row.get("title", "").strip(),
        "composers": composers,
        "lyricists": lyricists,
        "year": None,
        "genres": [],
        "lyrics": row.get("lyrics", ""),
        "urls": urls,
        "source": None,  # để null
        "note": ""
    })

df_new = pd.DataFrame(records)

# === Xuất ra file ===
df_new.to_csv("normalized_tkaraoke_standard_2.csv", index=False, encoding="utf-8-sig")

print("✅ Đã chuẩn hóa file 'normalize_tkaraoke_output.csv'")
print(df_new[["title", "composers", "lyricists"]].head(5))


In [5]:
import pandas as pd
import re
import ast

# === Đọc file gốc ===
df = pd.read_csv("normalize_tkaraoke_output.csv")

def ensure_list(x):
    if pd.isna(x) or x == "Null":
        return []
    if isinstance(x, list):
        return [str(v).strip() for v in x if str(v).strip()]
    if isinstance(x, str):
        # nếu là chuỗi chứa list literal, thử parse
        try:
            import ast
            val = ast.literal_eval(x)
            if isinstance(val, list):
                return [str(v).strip() for v in val if str(v).strip()]
        except:
            return [v.strip() for v in x.split(",") if v.strip()]
    return [str(x).strip()]

def extract_authors(text):
    """
    Trả về (composers:list[str], lyricists:list[str])
    - Xử lý được cả dạng:
        "Tác giả:NhạcAnh Bằng, thơThái Can"
        "Tác giả:NhạcNguyễn Hiền, lờiKim Tuấn"
        "Tác giả:Hoàng Thi Thơ"
    - Nhận diện Nhạc/Lời/Thơ dính liền hoặc có dấu cách
    - Không nhầm 'Thơ' trong tên người
    """
    composers, lyricists = [], []
    if not isinstance(text, str) or not text.strip():
        return [], []

    # 1. Bỏ prefix
    t = text.replace("Tác giả:", "").strip()

    # 2. Loại ký tự ẩn
    t = re.sub(r'[\u200B-\u200D\uFEFF\u00A0]', '', t)

    # 3. Thêm khoảng trắng sau các vai trò nếu dính liền (vd: 'NhạcAnh Bằng' -> 'Nhạc Anh Bằng')
    t = re.sub(r'(?i)(Nhạc|Lời|Thơ)(?=[A-ZÀ-Ỵa-zà-ỹ])', r'\1 ', t)

    # 4. Chuẩn hóa khoảng trắng
    t = re.sub(r'\s+', ' ', t).strip()

    # 5. Regex nhận diện khối vai trò
    # - Có thể có dấu ':' hoặc không
    # - Giới hạn vai trò chỉ match khi đứng đầu hoặc sau dấu ',' hoặc '&'
    pattern = re.compile(r'(?i)\b(Nhạc|Lời|Thơ)\b\s*:?\s*(.*?)(?=(?:\bNhạc\b|\bLời\b|\bThơ\b|$))')

    matches = list(pattern.finditer(t))
    split_re = re.compile(r'\s*(?:,|&|\bvà\b)\s*', flags=re.IGNORECASE)

    if matches:
        for m in matches:
            role = m.group(1).strip().lower()
            names_raw = m.group(2).strip()
            names = [n.strip() for n in split_re.split(names_raw) if n.strip()]
            if role.startswith('nhạc'):
                composers.extend(names)
            else:
                lyricists.extend(names)
    else:
        # không có vai trò -> toàn bộ là composer
        names = [n.strip() for n in split_re.split(t) if n.strip()]
        composers.extend(names)

    # Loại trùng, giữ thứ tự
    def unique(seq):
        seen, out = set(), []
        for x in seq:
            if x not in seen and x:
                seen.add(x)
                out.append(x)
        return out

    return unique(composers), unique(lyricists)



# === Tạo cấu trúc dữ liệu chuẩn ===
records = []
for _, row in df.iterrows():
    composers, lyricists = extract_authors(row.get("artist", ""))
    urls = ensure_list(row.get("urls", ""))

    records.append({
        "title": row.get("title", "").strip(),
        "composers": composers,
        "lyricists": lyricists,
        "year": None,
        "genres": [],
        "lyrics": row.get("lyrics", ""),
        "urls": urls,
        "source": None,  # để null
        "note": ""
    })

df_new = pd.DataFrame(records)

# === Xuất ra file ===
df_new.to_csv("normalized_tkaraoke_standard_2.csv", index=False, encoding="utf-8-sig")

print("✅ Đã chuẩn hóa file 'normalize_tkaraoke_output.csv'")
print(df_new[["title", "composers", "lyricists"]].head(5))


✅ Đã chuẩn hóa file 'normalize_tkaraoke_output.csv'
                                          title              composers  \
0                              Đắp Mộ Cuộc Tình             [Vũ Thanh]   
1                                  Bài Hát Xuân       [Trần Thanh Sơn]   
2                              Tình Ơi Hỡi Tình         [Eric Lê Phúc]   
3                                  Em Không Thể  [Touliver, Tiên Tiên]   
4  Về Đâu Mái Tóc Người Thương (Yêu Làn Tóc Ấy)            [Hoài Linh]   

            lyricists  
0                  []  
1                  []  
2  [Nguyễn Tất Nhiên]  
3         [Tiên Tiên]  
4                  []  


# chuẩn hóa từng cột theo yêu cầu : NHACVN và tkaraoke

CỘT TITLE

Quy chuan cho ten bai hat ở cot text:
tat ca cac ki tu viet thuong
khong co dau dac biet va dau cach & dau va cuoi
giua nhung chu chi co 1 dau cach
thay dau & bang chu va

In [26]:
# Đọc file CSV
df = pd.read_csv("normalized_tkaraoke_standard.csv")

# Hàm chuẩn hóa tiêu đề bài hát
def normalize_title(text):
    if not isinstance(text, str):
        return text
    text = text.lower()                              # 1. Viết thường
    text = text.replace('&', ' và ')                  # 2. Thay & → và
    text = re.sub(r'[^\w\s]', '', text, flags=re.UNICODE)  # 3. Giữ lại chữ cái có dấu và số
    text = re.sub(r'\s+', ' ', text)                  # 4. Chỉ 1 khoảng trắng giữa các từ
    text = text.strip()                               # 5. Bỏ khoảng trắng đầu/cuối
    return text

# Áp dụng cho cột title
df['title'] = df['title'].apply(normalize_title)

# Ghi đè hoặc lưu file mới
df.to_csv("normalized_tkaraoke_standard_clean.csv", index=False, encoding='utf-8-sig')

## ĐẾM SỐ DÒNG

In [ ]:
file_list = [
    "normalized_Nhacvn_standard.csv",
    "normalized_Nhacvn_standard_clean.csv",
    "normalized_tkaraoke_standard.csv",
    "normalized_tkaraoke_standard_clean.csv"
]

results = {}

print("Counting lines for the following CSV files:")
for filename in file_list:
    try:
        # Mở tệp và đếm số dòng hiệu quả
        with open(filename, 'r', encoding='utf-8', errors='ignore') as f:
            line_count = sum(1 for line in f)
        results[filename] = line_count
        print(f"- {filename}: {line_count} lines")
    except FileNotFoundError:
        results[filename] = "File not found"
        print(f"- {filename}: File not found (Lỗi: Không tìm thấy tệp)")
    except Exception as e:
        results[filename] = f"Error: {e}"
        print(f"- {filename}: Error occurred: {e} (Lỗi xảy ra: {e})")

print("\n--- Summary (Tóm tắt) ---")
for filename, count in results.items():
    print(f"{filename}: {count}")

# CHUẨN HÓA CỘT LYRICS

## NHACVN

In [34]:
import re
import pandas as pd

def clean_lyrics(text: str) -> str:
    if not isinstance(text, str):
        return ""

    # 1️⃣ Xóa ký tự ẩn như ZWNBSP, NBSP, BOM
    text = re.sub(r'[\u200B-\u200D\uFEFF\u00A0]', '', text)

    # 2️⃣ Xóa số thứ tự kiểu "1. ", "23. "
    text = re.sub(r'(^|\s)\d+\.\s*', r'\1', text)

    # 3️⃣ Xóa chỉ dẫn giọng hát và điệp khúc
    text = re.sub(
        r'\b(giọng\s*nam|giọng\s*nữ|nam|nữ|đk|điệp\s*khúc)\s*:\s*',
        '',
        text,
        flags=re.IGNORECASE
    )

    # 4️⃣ Xóa toàn bộ nội dung trong ngoặc tròn, ngoặc vuông nếu có
    text = re.sub(r'[\(\[].*?[\)\]]', '', text)

    # 5️⃣ Thay dấu & bằng chữ "và"
    text = text.replace('&', 'và')

    # 6️⃣ Chuyển toàn bộ sang chữ thường
    text = text.lower()

    # 7️⃣ Giữ lại chữ cái, số, khoảng trắng **và dấu phẩy**
    text = re.sub(r'[^a-zàáảãạăằắẳẵặâầấẩẫậèéẻẽẹêềếểễệ'
                  r'ìíỉĩịòóỏõọôồốổỗộơờớởỡợ'
                  r'ùúủũụưừứửữựỳýỷỹỵđ0-9,\s]', '', text)

    # 8️⃣ Chuẩn hóa khoảng trắng
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# === Chạy xử lý ===
df = pd.read_csv('normalized_Nhacvn_standard_clean.csv')

df['lyrics'] = df['lyrics'].astype(str).apply(clean_lyrics)

# Lưu kết quả
output_path = 'done_lyrics_nhacvn.csv'
df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"✅ Đã chuẩn hóa lyrics (bao gồm xóa 'ĐK:') & lưu tại '{output_path}'")


✅ Đã chuẩn hóa lyrics (bao gồm xóa 'ĐK:') & lưu tại 'done_lyrics_nhacvn.csv'


### TKARAOKE

In [35]:
import re
import pandas as pd

def clean_lyrics(text: str) -> str:
    if not isinstance(text, str):
        return ""

    # 1️⃣ Xóa ký tự ẩn như ZWNBSP, NBSP, BOM
    text = re.sub(r'[\u200B-\u200D\uFEFF\u00A0]', '', text)

    # 2️⃣ Xóa số thứ tự kiểu "1. ", "23. "
    text = re.sub(r'(^|\s)\d+\.\s*', r'\1', text)

    # 3️⃣ Xóa chỉ dẫn giọng hát và điệp khúc
    text = re.sub(
        r'\b(giọng\s*nam|giọng\s*nữ|nam|nữ|đk|điệp\s*khúc)\s*:\s*',
        '',
        text,
        flags=re.IGNORECASE
    )

    # 4️⃣ Xóa toàn bộ nội dung trong ngoặc tròn, ngoặc vuông nếu có
    text = re.sub(r'[\(\[].*?[\)\]]', '', text)

    # 5️⃣ Thay dấu & bằng chữ "và"
    text = text.replace('&', 'và')

    # 6️⃣ Chuyển toàn bộ sang chữ thường
    text = text.lower()

    # 7️⃣ Giữ lại chữ cái, số, khoảng trắng **và dấu phẩy**
    text = re.sub(r'[^a-zàáảãạăằắẳẵặâầấẩẫậèéẻẽẹêềếểễệ'
                  r'ìíỉĩịòóỏõọôồốổỗộơờớởỡợ'
                  r'ùúủũụưừứửữựỳýỷỹỵđ0-9,\s]', '', text)

    # 8️⃣ Chuẩn hóa khoảng trắng
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# === Chạy xử lý ===
df = pd.read_csv('normalized_tkaraoke_standard_clean.csv')

df['lyrics'] = df['lyrics'].astype(str).apply(clean_lyrics)

# Lưu kết quả
output_path = 'done_tkaraoke_nhacvn.csv'
df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"✅ Đã chuẩn hóa lyrics (bao gồm xóa 'ĐK:') & lưu tại '{output_path}'")


✅ Đã chuẩn hóa lyrics (bao gồm xóa 'ĐK:') & lưu tại 'done_tkaraoke_nhacvn.csv'


# Refill cột 'lyricists' , nếu trống thì lấy composers lấp qua

In [40]:
def fill_missing_lyricists(input_file_name, output_file_name):
    # Đọc file CSV
    df = pd.read_csv(input_file_name)

    # Đếm trước khi xử lý
    missing_before = df['lyricists'].isna().sum() + (df['lyricists'].astype(str).str.strip().isin(['', '[]'])).sum()
    print(f"Số dòng 'lyricists' bị trống hoặc thiếu trong {input_file_name}: {missing_before}")

    # Chuẩn hóa giá trị trống thành NaN
    df['lyricists'] = df['lyricists'].replace(['', '[]', '[ ]', 'nan', np.nan], np.nan)

    # Điền lyricists từ composers nếu lyricists đang NaN
    df['lyricists'] = df['lyricists'].fillna(df['composers'])

    # Đếm lại sau khi xử lý
    missing_after = df['lyricists'].isna().sum()
    print(f"Số dòng 'lyricists' còn thiếu sau khi điền: {missing_after}")

    # Lưu ra file CSV mới
    df.to_csv(output_file_name, index=False, encoding='utf-8-sig')
    print(f"✅ Đã lưu kết quả vào: {output_file_name}")


# === Ví dụ sử dụng ===
fill_missing_lyricists("done_lyrics_nhacvn.csv", "refill_lyricists_nhacvn.csv")
fill_missing_lyricists("done_tkaraoke_nhacvn.csv", "refill_lyricists_tkaraoke.csv")

Số dòng 'lyricists' bị trống hoặc thiếu trong done_lyrics_nhacvn.csv: 78785
Số dòng 'lyricists' còn thiếu sau khi điền: 0


/var/folders/dm/b6833525473fb0p2gk_x1dmm0000gp/T/ipykernel_3041/1941505324.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['lyricists'] = df['lyricists'].replace(['', '[]', '[ ]', 'nan', np.nan], np.nan)


✅ Đã lưu kết quả vào: refill_lyricists_nhacvn.csv
Số dòng 'lyricists' bị trống hoặc thiếu trong done_tkaraoke_nhacvn.csv: 46905
Số dòng 'lyricists' còn thiếu sau khi điền: 0
✅ Đã lưu kết quả vào: refill_lyricists_tkaraoke.csv


# Fill cột source cho từng trang

## Thống kê những thể loại nhạc nước ngoài NhacVN

In [34]:
# === Đọc dữ liệu ===
df = pd.read_csv("filled_source_tkaraoke.csv")

# === Gộp composer và lyricists lại để khảo sát dễ hơn ===
combined = df['composers'].fillna('') + ' | ' + df['lyricists'].fillna('')

# === Tìm các cụm có dạng "Nhạc ...", "Nhạc ... Lời Việt" ===
pattern = r"Ngoại[^,;\]\[]*"
matches = []

for text in combined:
    found = re.findall(pattern, str(text))
    if found:
        matches.extend(found)

# === Đưa vào DataFrame thống kê tần suất xuất hiện ===
survey = pd.Series(matches).value_counts().reset_index()
survey.columns = ['pattern_found', 'count']

# === In ra kết quả khảo sát ===
print("Các dạng 'Nhạc ...' tìm thấy trong composers/lyricists:")
display(survey.head(100))  # hiển thị top 50 kết quả
output = survey.to_csv("hello.csv")


Các dạng 'Nhạc ...' tìm thấy trong composers/lyricists:


,pattern_found,count
0,Ngoại',2534
1,Ngoại (Trung Hoa)',776
2,Ngoại,488
3,Ngoại (Trung Hoa),150
4,Ngoại (Pháp)',136
...,...,...
63,Ngoại (Trung Hoa',1
64,Ngoại: Johnny Mandel,1
65,Ngoại (Đan Mạch),1
66,Ngoại (Áo),1


In [23]:
#chuẩn hóa lại

In [28]:
import pandas as pd
import re
from tqdm import tqdm
import ast

# === Đọc dữ liệu ===
df = pd.read_csv("filled_source_nhacvn.csv")

# === Danh sách cụm từ cần nhận dạng ===
music_keywords = [
    "Nhạc Hoa Lời Việt", "Nhạc Ngoại Lời Việt", "Nhạc Nước Ngoài", "Nhạc Pháp Lời Việt",
    "Nhạc Nhật Lời Việt", "Nhạc Hàn Lời Việt", "Nhạc nước ngoài", "Nhạc Nga Lời Việt",
    "Nhạc Thái Lời Việt", "Nhạc Hong Kong", "Nhạc Pháp", "Nhạc Hải Ngoại",
    "Nhạc Thái Lan Lời Việt", "Nhạc sĩ)", "Nhạc Dân Gian", "Nhạc ngoại LV",
    "Nhạc Nga", "Nhạc ngoại", "Nhạc Hàn Quốc", "Nhạc phim", "Nhạc Đạo",
    "Nhạc Nước Ngoài Lời Việt", "Nhạc Nguyễn Kế Khuyến", "Nhạc Ý", "Nhạc Trung Quốc",
    "Nhạc Chèo", "Nhạc Anh", "Nhạc Hàn", "Nhạc Trong Phim",
]

# === Danh sách giá trị "vô nghĩa" cần xóa luôn ===
unknown_terms = {"unknown", "Unknown", "Không rõ", "?", "none", "None", "NaN"}

# === Chuẩn bị cột genres (đảm bảo là list) ===
if "genres" not in df.columns:
    df["genres"] = [[] for _ in range(len(df))]
else:
    df["genres"] = df["genres"].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith("[") else ([] if pd.isna(x) else [x])
    )

# === Hàm chuẩn hóa từng ô (dạng list hoặc chuỗi) ===
def normalize_field(value):
    """Làm sạch giá trị của composer/lyricist"""
    found = []

    # 1️⃣ Nếu là NaN hoặc None
    if pd.isna(value):
        return "", []

    # 2️⃣ Nếu là list được lưu dạng string
    if isinstance(value, str) and value.startswith("["):
        try:
            value = ast.literal_eval(value)
        except Exception:
            value = [value]

    # 3️⃣ Nếu là list thật
    if isinstance(value, list):
        # Bỏ Unknown và các mục vô nghĩa
        clean_items = []
        for item in value:
            if not isinstance(item, str):
                continue
            item = item.strip()
            if item.lower() in [t.lower() for t in unknown_terms]:
                continue  # bỏ qua Unknown
            # tìm cụm "Nhạc ..."
            matched = [kw for kw in music_keywords if re.search(re.escape(kw), item, re.IGNORECASE)]
            if matched:
                found.extend(matched)
            else:
                clean_items.append(item)
        new_value = ", ".join(clean_items)
    else:
        # 4️⃣ Nếu là chuỗi đơn
        text = value.strip()
        if text.lower() in [t.lower() for t in unknown_terms]:
            return "", []
        new_value = text
        for kw in music_keywords:
            if re.search(re.escape(kw), new_value, re.IGNORECASE):
                found.append(kw)
                new_value = re.sub(re.escape(kw), "", new_value, flags=re.IGNORECASE)

        new_value = re.sub(r"\s*[,;|]+\s*", ", ", new_value.strip())
        new_value = re.sub(r"\s{2,}", " ", new_value).strip(" ,;|")

    return new_value.strip(), found

# === Áp dụng cho toàn bộ dữ liệu ===
for idx in tqdm(range(len(df)), desc="Chuẩn hóa composers & lyricists"):
    composer = df.at[idx, "composers"]
    lyricist = df.at[idx, "lyricists"]

    new_composer, genres1 = normalize_field(composer)
    new_lyricist, genres2 = normalize_field(lyricist)

    df.at[idx, "composers"] = new_composer
    df.at[idx, "lyricists"] = new_lyricist
    df.at[idx, "genres"] = sorted(set(df.at[idx, "genres"] + genres1 + genres2))

# === Xuất file kết quả ===
output_file = "l3/normalized_v3_no_unknown.csv"
df.to_csv(output_file, index=False, encoding="utf-8-sig")
print(f"✅ Đã ghi file: {output_file}")


Chuẩn hóa composers & lyricists: 100%|██████████| 78785/78785 [01:02<00:00, 1265.17it/s]


✅ Đã ghi file: l3/normalized_v3_no_unknown.csv


# NHACVN

In [ ]:
import pandas as pd
import re

# === Đọc file đầu vào ===
input_file = "refill_lyricists_nhacvn.csv"  # 👈 đổi tên nếu cần
df = pd.read_csv(input_file)

# === Hàm nhận diện quốc gia dựa trên composers + lyricists ===
def detect_source(row):
    text = str(row['composers']) + ' ' + str(row['lyricists'])
    text = text.lower()

    mapping = {
        'hoa': 'Trung Quốc',
        'trung quốc': 'Trung Quốc',
        'hong kong': 'Trung Quốc',
        'hàn': 'Hàn Quốc',
        'hàn quốc': 'Hàn Quốc',
        'nhật': 'Nhật Bản',
        'pháp': 'Pháp',
        'nga': 'Nga',
        'anh': 'Anh',
        'mỹ': 'Mỹ',
        'ý': 'Ý',
        'thái': 'Thái Lan',
        'thái lan': 'Thái Lan',
        'nước ngoài': 'Nhạc nước ngoài',
        'ngoại': 'Nhạc nước ngoài',
        # 'hải ngoại' KHÔNG được tính là nước ngoài
    }

    # Ưu tiên loại trừ "nhạc hải ngoại" => Việt Nam
    if re.search(r"nhạc\s*hải\s*ngoại", text):
        return "Việt Nam"

    # Dò các từ khóa quốc gia khác
    for key, value in mapping.items():
        if re.search(rf"nhạc\s*{key}", text):
            return value

    return 'Việt Nam'  # Mặc định nếu không khớp

# === Áp dụng vào DataFrame ===
df['source'] = df.apply(detect_source, axis=1)

# === Xuất file kết quả ===
output_file = "filled_source_nhacvn.csv"
df.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"✅ Đã tạo file: {output_file} (Nhạc Hải Ngoại được tính là Việt Nam)")

# === Thống kê nhanh ===
print("\n📊 Thống kê giá trị trong cột 'source':")
print(df['source'].value_counts())

# TKARAOKE

In [29]:
import pandas as pd
import re

# === Đọc dữ liệu ===
df = pd.read_csv("filled_source_nhacvn.csv")

# === Gộp composer và lyricists lại để khảo sát ===
combined = df['composers'].fillna('') + ' | ' + df['lyricists'].fillna('')

# === Tìm tất cả cụm có chứa “Ngoại” hoặc “Ngoai” ===
pattern = r"[Nn]go[aạ]i[^\]\|,;]*"
matches = []

for text in combined:
    found = re.findall(pattern, str(text))
    if found:
        matches.extend(found)

# === Thống kê tần suất xuất hiện ===
survey = pd.Series(matches).value_counts().reset_index()
survey.columns = ['pattern_found', 'count']

# === Lưu file khảo sát ===
output_file = "survey_source_tkaraoke.csv"
survey.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"✅ Đã lưu kết quả khảo sát vào file: {output_file}")
print("📊 Các dạng 'Ngoại' / 'Ngoai' tìm thấy trong composers hoặc lyricists:")
display(survey.head(69))


✅ Đã lưu kết quả khảo sát vào file: survey_source_tkaraoke.csv
📊 Các dạng 'Ngoại' / 'Ngoai' tìm thấy trong composers hoặc lyricists:


,pattern_found,count
0,Ngoại Lời Việt',1710
1,Ngoại)',1292
2,Ngoại',12
3,ngoại LV',8
4,ngoại',4


In [36]:
import pandas as pd
import re
import ast
from tqdm import tqdm

# === Đọc dữ liệu TKaraoke ===
df = pd.read_csv("filled_source_tkaraoke.csv")

# === Các cụm nhận diện "Nhạc Ngoại" trong TKaraoke ===
foreign_keywords = [
    "Ngoại", "Ngoại Quốc", "Ngoại.", "Ngoại '", "Ngoại '", "Ngoại -",
    "Ngoại (Trung Hoa)", "Ngoại (Trung Quốc)", "Ngoại (Pháp)", "Ngoại (Anh)",
    "Ngoại (Nga)", "Ngoại (Hàn Quốc)", "Ngoại (Nhật)", "Ngoại (Thái Lan)",
    "Ngoại (Ý)", "Ngoại (Tây Ban Nha)", "Ngoại (Ấn Độ)", "Ngoại (Đức)",
    "Ngoại (Đài Loan)", "Ngoại (Mỹ)", "Ngoại (Hoa)", "Ngoại (Columbia)",
    "Ngoại (Triều Tiên)", "Ngoại (Indonexia)", "Ngoại (Malaysia)",
    "Ngoại (Philipin)", "Ngoại (Bồ Đào Nha)", "Ngoại (Hà Lan)",
    "Ngoại (Áo)", "Ngoại (Đan Mạch)", "Ngoại (Argentina)", "Ngoại (Quốc)",
    "Ngoại Phi Tiên", "Ngoại KimNguyệt Trần Lê Tú", "Ngoại (Antonio Vivaldi)"
]

# === Các giá trị vô nghĩa cần xóa hẳn ===
unknown_terms = {"unknown", "Unknown", "Không rõ", "?", "none", "None", "NaN","Chưa Biết", "Chư Biết"}

# === Đảm bảo cột genres tồn tại và là list ===
if "genres" not in df.columns:
    df["genres"] = [[] for _ in range(len(df))]
else:
    df["genres"] = df["genres"].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith("[") else ([] if pd.isna(x) else [x])
    )

# === Hàm xử lý từng ô ===
def normalize_tkaraoke_field(value):
    """Làm sạch composer/lyricist cho dữ liệu TKaraoke"""
    found = []

    if pd.isna(value):
        return "", []

    # Nếu là list lưu dạng string
    if isinstance(value, str) and value.startswith("["):
        try:
            value = ast.literal_eval(value)
        except Exception:
            value = [value]

    # Nếu là list thật
    if isinstance(value, list):
        clean_items = []
        for item in value:
            if not isinstance(item, str):
                continue
            item = item.strip()
            if item.lower() in [t.lower() for t in unknown_terms]:
                continue  # loại Unknown
            # nhận diện "Ngoại..."
            matched = [kw for kw in foreign_keywords if re.search(re.escape(kw), item, re.IGNORECASE)]
            if matched:
                found.extend(matched)
            else:
                clean_items.append(item)
        new_value = ", ".join(clean_items)
    else:
        # Chuỗi thường
        text = value.strip()
        if text.lower() in [t.lower() for t in unknown_terms]:
            return "", []
        new_value = text
        for kw in foreign_keywords:
            if re.search(re.escape(kw), new_value, re.IGNORECASE):
                found.append(kw)
                new_value = re.sub(re.escape(kw), "", new_value, flags=re.IGNORECASE)

        new_value = re.sub(r"\s*[,;|]+\s*", ", ", new_value.strip())
        new_value = re.sub(r"\s{2,}", " ", new_value).strip(" ,;|")

    return new_value.strip(), found

# === Áp dụng cho toàn bộ dữ liệu ===
for idx in tqdm(range(len(df)), desc="Chuẩn hóa TKaraoke composers & lyricists"):
    composer = df.at[idx, "composers"]
    lyricist = df.at[idx, "lyricists"]

    new_composer, genres1 = normalize_tkaraoke_field(composer)
    new_lyricist, genres2 = normalize_tkaraoke_field(lyricist)

    df.at[idx, "composers"] = new_composer
    df.at[idx, "lyricists"] = new_lyricist
    df.at[idx, "genres"] = sorted(set(df.at[idx, "genres"] + genres1 + genres2))

# === Xuất file kết quả ===
output_file = "l3/normalized_tkaraoke_no_unknown.csv"
df.to_csv(output_file, index=False, encoding="utf-8-sig")
print(f"✅ Đã ghi file: {output_file}")

Chuẩn hóa TKaraoke composers & lyricists: 100%|██████████| 49296/49296 [00:16<00:00, 3055.67it/s]


✅ Đã ghi file: l3/normalized_tkaraoke_no_unknown.csv


In [14]:
import pandas as pd
import re

# === Đọc file đầu vào ===
input_file = "refill_lyricists_tkaraoke.csv"
df = pd.read_csv(input_file)

# === Hàm nhận diện quốc gia hoặc vùng theo nội dung composers + lyricists ===
def detect_source(row):
    text = str(row['composers']) + ' ' + str(row['lyricists'])
    text = text.lower()

    # Bảng quy đổi các nước và vùng lãnh thổ thường gặp
    mapping = {
        # Đông Á
        'trung hoa': 'Trung Quốc',
        'trung quốc': 'Trung Quốc',
        'trung': 'Trung Quốc',
        'đài loan': 'Đài Loan',
        'hong kong': 'Trung Quốc',
        'triều tiên': 'Triều Tiên',
        'hàn quốc': 'Hàn Quốc',
        'hàn': 'Hàn Quốc',
        'nhật': 'Nhật Bản',

        # Đông Nam Á
        'thái lan': 'Thái Lan',
        'indonesia': 'Indonesia',
        'indonexia': 'Indonesia',
        'malaysia': 'Malaysia',
        'philippin': 'Philippines',

        # Tây Á, Nam Á
        'ấn độ': 'Ấn Độ',

        # Châu Âu
        'pháp': 'Pháp',
        'anh': 'Anh',
        'đức': 'Đức',
        'tây ban nha': 'Tây Ban Nha',
        'ý': 'Ý',
        'áo': 'Áo',
        'bồ đào nha': 'Bồ Đào Nha',
        'thụy điển': 'Thụy Điển',
        'đan mạch': 'Đan Mạch',

        # Châu Mỹ
        'mỹ': 'Mỹ',
        'cu ba': 'Cuba',
        'columbia': 'Colombia',
        'argentina': 'Argentina',
        'mexico': 'Mexico',

        # Ngoại quốc nói chung
        'ngoại quốc': 'Nhạc nước ngoài',
    }

    # === Dò các từ khóa trong nội dung ===
    for key, value in mapping.items():
        # Nhận dạng cả các dạng “Ngoại (Pháp)”, “Ngoại(Trung)”, “Ngoại : Johnny Mandel”
        if re.search(rf"ngo[aạ]i[^a-zA-Z0-9]*{key}", text):
            return value

    # === Nếu có “Ngoại” nhưng không rõ quốc gia nào ===
    if re.search(r"ngo[aạ]i", text):
        return "Nhạc nước ngoài"

    # === Ngược lại, mặc định là Việt Nam ===
    return "Việt Nam"


# === Áp dụng hàm nhận diện ===
df['source'] = df.apply(detect_source, axis=1)

# === Xuất kết quả ===
output_file = "filled_source_tkaraoke.csv"
df.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"✅ Đã tạo file: {output_file}")

# === Thống kê nhanh ===
print("\n📊 Thống kê giá trị cột 'source':")
print(df['source'].value_counts())


✅ Đã tạo file: filled_source_tkaraoke.csv

📊 Thống kê giá trị cột 'source':
source
Việt Nam           46501
Nhạc nước ngoài     1881
Trung Quốc           545
Pháp                 139
Anh                   71
Nhật Bản              53
Hàn Quốc              50
Thái Lan              17
Tây Ban Nha            6
Ý                      6
Thụy Điển              4
Áo                     3
Ấn Độ                  3
Mexico                 2
Đài Loan               2
Mỹ                     2
Đức                    2
Indonesia              2
Bồ Đào Nha             1
Triều Tiên             1
Đan Mạch               1
Argentina              1
Colombia               1
Cuba                   1
Malaysia               1
Name: count, dtype: int64


# Merge

Quy trình merge như sau:
- Đầu tiên nối 2 file lại với nhau
- Filter trong cột 'lyrics', nếu có dòng nào trùng nhau , chọn giữ lại dòng có nhiều dữ liệu nhất(case 1:nếu dữ liệu duplicate hoàn toàn, giữ lại dòng đầu tiên; case 2: nếu có số dữ liệu bằng nhau, giữ lại những dòng đó)
- cột genres: nếu có dòng nào trùng nhau, gôm danh sách thể loại lại với nhau, loại bỏ trùng lặp trong danh sách
- urls : lấy tất cả url của những dòng bị trùng
-

In [ ]:
#Noi file va detect
import pandas as pd
import ast

# Hàm normalize lyrics
def normalize_lyrics(text):
    """Chuẩn hóa lyrics để so sánh"""
    if pd.isna(text) or text == '':
        return ''

    text = str(text)
    # 1. Loại bỏ khoảng trắng đầu/cuối
    text = text.strip()
    # 2. Chuyển thành chữ thường
    text = text.lower()
    # 3. Thay '&' → 'và'
    text = text.replace('&', 'và')
    # 4. Chuẩn hóa nhiều space → 1 space
    text = ' '.join(text.split())

    return text

# Đọc 2 files
print("=" * 80)
print("ĐANG ĐỌC FILES...")
print("=" * 80)

df1 = pd.read_csv('filled_source_nhacvn.csv')
df2 = pd.read_csv('filled_source_tkaraoke.csv')

# Thêm metadata
df1['_source_file'] = 'nhacvn'
df1['_original_index'] = df1.index
df2['_source_file'] = 'tkaraoke'
df2['_original_index'] = df2.index

print(f"✓ File nhacvn: {len(df1)} dòng")
print(f"✓ File tkaraoke: {len(df2)} dòng")
print()

# Nối 2 files
combined = pd.concat([df1, df2], ignore_index=True)
print(f"✓ Tổng số dòng sau khi nối: {len(combined)}")
print()

# Thêm cột normalized lyrics
combined['_normalized_lyrics'] = combined['lyrics'].apply(normalize_lyrics)

# Loại bỏ các dòng có lyrics rỗng
combined_filtered = combined[combined['_normalized_lyrics'] != ''].copy()
print(f"✓ Số dòng có lyrics: {len(combined_filtered)}")
print()

# Group by normalized lyrics
print("=" * 80)
print("ĐANG DETECT DUPLICATES...")
print("=" * 80)

grouped = combined_filtered.groupby('_normalized_lyrics')

# Lọc các groups có > 1 dòng (duplicates)
duplicate_groups = []
for normalized_lyrics, group in grouped:
    if len(group) > 1:
        duplicate_groups.append({
            'normalized_lyrics': normalized_lyrics,
            'count': len(group),
            'data': group.sort_values(['_source_file', '_original_index'])
        })

print(f"\n🔍 Tìm thấy {len(duplicate_groups)} nhóm trùng lặp")
print(f"📊 Tổng số dòng bị trùng: {sum(g['count'] for g in duplicate_groups)}")
print()

# Hiển thị chi tiết từng group
if len(duplicate_groups) > 0:
    print("=" * 80)
    print("CHI TIẾT CÁC NHÓM TRÙNG LẶP")
    print("=" * 80)

    for idx, group in enumerate(duplicate_groups, 1):
        print(f"\n{'=' * 80}")
        print(f"NHÓM #{idx} - {group['count']} dòng trùng lặp")
        print(f"{'=' * 80}")

        for row_idx, (_, row) in enumerate(group['data'].iterrows(), 1):
            print(f"\n--- Dòng {row_idx} [{row['_source_file'].upper()}] ---")
            print(f"Title: {row['title']}")
            print(f"Composers: {row['composers']}")
            print(f"Lyricists: {row['lyricists']}")
            print(f"Year: {row['year']}")
            print(f"Genres: {row['genres']}")
            print(f"Source: {row['source']}")
            print(f"URLs: {row['urls']}")
            print(f"Note: {row['note']}")
            print(f"Lyrics preview: {str(row['lyrics'])[:150]}...")
            print(f"Original index in file: {row['_original_index']}")

        print(f"\n{'-' * 80}")

else:
    print("=" * 80)
    print("✅ KHÔNG CÓ DUPLICATES!")
    print("=" * 80)
    print("Tất cả lyrics đều unique.")

print("\n" + "=" * 80)
print("HOÀN TẤT!")
print("=" * 80)

In [ ]:
import pandas as pd
import ast
import numpy as np

def parse_list_field(field):
    """Parse list field từ string format"""
    if pd.isna(field) or field == '' or field == '[]':
        return []
    try:
        # Nếu đã là list thì return luôn
        if isinstance(field, list):
            return field
        # Parse string format ['item1', 'item2']
        result = ast.literal_eval(field)
        return result if isinstance(result, list) else [result]
    except:
        return [str(field)]

def merge_list_fields(values):
    """Gộp nhiều list lại, loại trùng, giữ thứ tự"""
    all_items = []
    for val in values:
        items = parse_list_field(val)
        for item in items:
            if item and item not in all_items:
                all_items.append(item)
    return all_items

def select_best_title(titles):
    """Chọn title dài nhất và đầy đủ nhất"""
    valid_titles = [t for t in titles if pd.notna(t) and t != '']
    if not valid_titles:
        return ''
    # Chọn title dài nhất
    return max(valid_titles, key=len)

def select_best_year(years):
    """Chọn year hợp lệ đầu tiên (non-null)"""
    valid_years = [y for y in years if pd.notna(y) and y != '']
    return valid_years[0] if valid_years else np.nan

def merge_sources(sources):
    """Gộp sources, ưu tiên cụ thể hơn"""
    valid_sources = [s for s in sources if pd.notna(s) and s != '']
    if not valid_sources:
        return ''
    # Loại trùng
    unique_sources = []
    for s in valid_sources:
        if s not in unique_sources:
            unique_sources.append(s)
    # Nếu chỉ có 1 loại thì return
    if len(unique_sources) == 1:
        return unique_sources[0]
    # Nếu có nhiều loại khác nhau thì gộp
    return ' | '.join(unique_sources)

def merge_notes(notes):
    """Gộp notes nếu khác nhau"""
    valid_notes = [n for n in notes if pd.notna(n) and n != '']
    if not valid_notes:
        return ''
    # Loại trùng
    unique_notes = []
    for n in valid_notes:
        if n not in unique_notes:
            unique_notes.append(n)
    return ' | '.join(unique_notes)

def merge_group(group_df):
    """Merge một nhóm duplicate thành 1 dòng"""

    # Title: chọn dài nhất
    merged_title = select_best_title(group_df['title'].tolist())

    # Composers: gộp tất cả, loại trùng
    merged_composers = merge_list_fields(group_df['composers'].tolist())

    # Lyricists: gộp tất cả, loại trùng
    merged_lyricists = merge_list_fields(group_df['lyricists'].tolist())

    # Year: lấy giá trị hợp lệ đầu tiên
    merged_year = select_best_year(group_df['year'].tolist())

    # Genres: gộp tất cả, loại trùng
    merged_genres = merge_list_fields(group_df['genres'].tolist())

    # Lyrics: giữ nguyên (đã giống nhau rồi)
    merged_lyrics = group_df['lyrics'].iloc[0]

    # URLs: gộp tất cả
    merged_urls = merge_list_fields(group_df['urls'].tolist())

    # Source: gộp hoặc giữ cụ thể hơn
    merged_source = merge_sources(group_df['source'].tolist())

    # Note: gộp nếu khác nhau
    merged_note = merge_notes(group_df['note'].tolist())

    return pd.Series({
        'title': merged_title,
        'composers': merged_composers,
        'lyricists': merged_lyricists,
        'year': merged_year,
        'genres': merged_genres,
        'lyrics': merged_lyrics,
        'urls': merged_urls,
        'source': merged_source,
        'note': merged_note
    })

print("=" * 80)
print("BẮT ĐẦU MERGE DUPLICATES")
print("=" * 80)

# Lấy data từ cell trước (combined_filtered và grouped)
# Nếu chưa có thì phải chạy lại cell 1

# Tạo dataframe kết quả
merged_rows = []
unique_rows = []

total_groups = 0
total_duplicates = 0

for normalized_lyrics, group in grouped:
    if len(group) > 1:
        # Duplicate group - merge
        total_groups += 1
        total_duplicates += len(group)
        merged_row = merge_group(group)
        merged_rows.append(merged_row)
    else:
        # Unique row - giữ nguyên
        row = group.iloc[0]
        unique_rows.append({
            'title': row['title'],
            'composers': parse_list_field(row['composers']),
            'lyricists': parse_list_field(row['lyricists']),
            'year': row['year'],
            'genres': parse_list_field(row['genres']),
            'lyrics': row['lyrics'],
            'urls': parse_list_field(row['urls']),
            'source': row['source'],
            'note': row['note']
        })

print(f"\n✓ Đã merge {total_groups} nhóm duplicate ({total_duplicates} dòng)")
print(f"✓ Giữ nguyên {len(unique_rows)} dòng unique")

# Gộp merged và unique
result_df = pd.concat([
    pd.DataFrame(merged_rows),
    pd.DataFrame(unique_rows)
], ignore_index=True)

print(f"\n✓ Tổng số dòng sau merge: {len(result_df)}")
print(f"✓ Giảm được: {len(combined_filtered) - len(result_df)} dòng")

# Format lại list fields thành string để lưu CSV
result_df['composers'] = result_df['composers'].apply(lambda x: str(x) if x else '[]')
result_df['lyricists'] = result_df['lyricists'].apply(lambda x: str(x) if x else '[]')
result_df['genres'] = result_df['genres'].apply(lambda x: str(x) if x else '[]')
result_df['urls'] = result_df['urls'].apply(lambda x: str(x) if x else '[]')

# Sắp xếp lại cột theo thứ tự mong muốn
column_order = ['title', 'composers', 'lyricists', 'year', 'genres', 'lyrics', 'urls', 'source', 'note']
result_df = result_df[column_order]

# Lưu file
output_file = 'merged_nhacvnAndTkaraoke_result.csv'
result_df.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"\n✓ Đã lưu file: {output_file}")
print("=" * 80)

# Hiển thị mẫu kết quả merge
print("\nMẪU KẾT QUẢ MERGE:")
print("=" * 80)

# Lấy 1 ví dụ từ group đã merge
sample_merged = pd.DataFrame(merged_rows).head(3)
for idx, row in sample_merged.iterrows():
    print(f"\nDòng {idx + 1}:")
    print(f"  Title: {row['title']}")
    print(f"  Composers: {row['composers']}")
    print(f"  Lyricists: {row['lyricists']}")
    print(f"  Year: {row['year']}")
    print(f"  Genres: {row['genres']}")
    print(f"  URLs count: {len(row['urls'])}")
    print(f"  Source: {row['source']}")
    print(f"  Note: {row['note']}")
    print(f"  Lyrics preview: {str(row['lyrics'])[:100]}...")
    print("-" * 80)

print("\n✅ HOÀN TẤT MERGE!")

Fill year cho cho data

In [4]:
import pandas as pd
import requests
import time
import re
import logging
from tqdm import tqdm
from datetime import datetime
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from functools import lru_cache
import threading

# ==================== LOGGING CONFIG ====================
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)-8s | %(message)s",
    datefmt="%H:%M:%S"
)
logger = logging.getLogger(__name__)
start_time = datetime.now()
logger.info("=== Bắt đầu điền cột 'year' sử dụng MusicBrainz + Wikipedia + iTunes ===")

# ==================== FILE I/O ====================
input_file = "output/merged_yearFilled_final_2.csv"
output_file = "output/merged_yearFilled_final_3.csv"
checkpoint_file = "output/checkpoint_years.csv"

# Thread-safe lock cho việc ghi file
file_lock = threading.Lock()

# ==================== ĐỌC DỮ LIỆU & CHECKPOINT ====================
def load_data():
    """Đọc dữ liệu từ checkpoint hoặc file gốc"""
    # Kiểm tra checkpoint trước
    if os.path.exists(checkpoint_file):
        logger.info(f"🔄 Tìm thấy checkpoint! Đang đọc từ: {checkpoint_file}")
        df = pd.read_csv(checkpoint_file, encoding="utf-8-sig")
        df.columns = df.columns.str.strip().str.lower()
        logger.info(f"✅ Đã tải checkpoint với {len(df)} hàng")
        return df, True

    # Nếu không có checkpoint, đọc file gốc
    try:
        df = pd.read_csv(input_file, encoding="utf-8-sig")
        df.columns = df.columns.str.strip().str.lower()
        logger.info(f"Đã đọc {len(df)} hàng từ file: {input_file}")
        return df, False
    except Exception as e:
        logger.error(f"Lỗi khi đọc file CSV: {e}")
        raise SystemExit(e)

df, from_checkpoint = load_data()

# Đảm bảo các cột cần thiết tồn tại
for col in ["year", "note"]:
    if col not in df.columns:
        df[col] = ""
df["note"] = df["note"].astype(str)

# ==================== HÀM 1: MusicBrainz ====================
@lru_cache(maxsize=1000)
def get_song_year_musicbrainz(title, artist=None):
    try:
        query = title
        if artist:
            query += f" AND artist:{artist}"

        url = f"https://musicbrainz.org/ws/2/recording/?query={query}&fmt=json&limit=1"
        headers = {"User-Agent": "NhacVN-YearFiller/1.0 (example@gmail.com)"}
        r = requests.get(url, headers=headers, timeout=10)
        r.raise_for_status()
        data = r.json()

        recordings = data.get("recordings", [])
        if not recordings:
            return None

        rec = recordings[0]
        releases = rec.get("releases", [])
        if not releases:
            return None

        date = releases[0].get("date", "")
        if not date:
            return None

        match = re.search(r"\b(19|20)\d{2}\b", date)
        if match:
            return int(match.group(0))
        return None
    except Exception as e:
        # Chỉ log error quan trọng
        if "timeout" in str(e).lower():
            logger.debug(f"[MusicBrainz] Timeout: {title}")
        return None

# ==================== HÀM 2: Wikipedia ====================
@lru_cache(maxsize=1000)
def get_song_year_wikipedia(title):
    try:
        headers = {
            "User-Agent": "NhacVN-YearFiller/1.0 (https://github.com/duynguyen or contact@example.com)"
        }

        base_urls = [
            "https://vi.wikipedia.org/w/api.php",
            "https://en.wikipedia.org/w/api.php",
        ]

        for base_url in base_urls:
            params = {
                "action": "query",
                "list": "search",
                "srsearch": title,
                "format": "json",
                "utf8": 1,
                "srlimit": 1,
            }
            r = requests.get(base_url, params=params, headers=headers, timeout=10)
            if r.status_code == 403:
                continue

            r.raise_for_status()
            data = r.json()
            search_results = data.get("query", {}).get("search", [])
            if not search_results:
                continue

            page_title = search_results[0]["title"]
            extract_url = f"{base_url.replace('/w/api.php', '')}/api/rest_v1/page/summary/{page_title}"
            r = requests.get(extract_url, headers=headers, timeout=10)
            if r.status_code == 403:
                continue
            r.raise_for_status()
            summary_data = r.json()
            extract = summary_data.get("extract", "")

            match = re.search(r"\b(19|20)\d{2}\b", extract)
            if match:
                return int(match.group(0))

        return None
    except Exception as e:
        logger.debug(f"[Wikipedia] Error: {title}")
        return None

# ==================== HÀM 3: iTunes (FIX LỖI 404) ====================
@lru_cache(maxsize=1000)
def get_song_year_itunes(title, artist=None):
    try:
        query = title
        if artist:
            query += f" {artist}"
        url = "https://itunes.apple.com/search"
        params = {"term": query, "entity": "song", "limit": 1, "country": "us"}
        r = requests.get(url, params=params, timeout=10)

        # FIX: Không raise error cho 404, chỉ return None
        if r.status_code == 404:
            return None

        r.raise_for_status()
        data = r.json()

        results = data.get("results", [])
        if not results:
            return None

        release_date = results[0].get("releaseDate", "")
        if not release_date:
            return None

        match = re.search(r"\b(19|20)\d{2}\b", release_date)
        if match:
            return int(match.group(0))
        return None
    except requests.exceptions.RequestException as e:
        # Chỉ log lỗi nghiêm trọng, bỏ qua 404
        if "404" not in str(e):
            logger.debug(f"[iTunes] Error: {title}")
        return None
    except Exception as e:
        return None

# ==================== HÀM XỬ LÝ 1 DÒNG ====================
def process_single_row(idx, title, artist):
    """Xử lý 1 bài hát và trả về kết quả"""
    # Ưu tiên 1: MusicBrainz
    year = get_song_year_musicbrainz(title, artist)
    source = "MusicBrainz"

    # Ưu tiên 2: Wikipedia
    if not year:
        time.sleep(0.2)
        year = get_song_year_wikipedia(title)
        source = "Wikipedia" if year else None

    # Ưu tiên 3: iTunes
    if not year:
        time.sleep(0.2)
        year = get_song_year_itunes(title, artist)
        source = "iTunes" if year else None

    # Trả về kết quả
    if year:
        return idx, year, f"đã fill 'year' sử dụng {source}"
    else:
        return idx, None, "không tìm thấy thông tin năm phát hành"

# ==================== QUY TRÌNH FILL DỮ LIỆU (ĐA LUỒNG) ====================
need_fill = df["year"].isna() | (df["year"].astype(str).str.strip() == "")
indices = df[need_fill].index.tolist()
logger.info(f"Cần điền 'year' cho {len(indices)} bài hát")

if len(indices) == 0:
    logger.info("✅ Tất cả dữ liệu đã được điền đầy đủ!")
else:
    # Cấu hình đa luồng (tăng số workers để chạy nhanh hơn)
    MAX_WORKERS = 10  # Có thể tăng lên 15-20 nếu mạng ổn định
    checkpoint_interval = 100  # Lưu checkpoint mỗi 100 dòng

    logger.info(f"🚀 Sử dụng {MAX_WORKERS} threads để tăng tốc...")

    processed_count = 0

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        # Tạo danh sách tasks
        futures = {}
        for idx in indices:
            title = df.loc[idx, "title"]
            artist = None
            if "composers" in df.columns and isinstance(df.loc[idx, "composers"], str):
                artist = df.loc[idx, "composers"].split(",")[0]

            future = executor.submit(process_single_row, idx, title, artist)
            futures[future] = idx

        # Xử lý kết quả với progress bar
        for future in tqdm(as_completed(futures), total=len(indices), desc="Filling year"):
            idx, year, note = future.result()

            # Cập nhật DataFrame
            if year:
                df.loc[idx, "year"] = year
            df.loc[idx, "note"] = note

            processed_count += 1

            # Lưu checkpoint định kỳ (thread-safe)
            if processed_count % checkpoint_interval == 0:
                with file_lock:
                    df.to_csv(checkpoint_file, index=False, encoding="utf-8-sig")
                    logger.info(f"💾 Đã lưu checkpoint tại dòng {processed_count}/{len(indices)}")

# ==================== GHI FILE CUỐI ====================
df.to_csv(output_file, index=False, encoding="utf-8-sig")
logger.info(f"✅ Đã ghi file kết quả: {output_file}")

if os.path.exists(checkpoint_file):
    os.remove(checkpoint_file)
    logger.info("🗑️ Đã xóa checkpoint file")

elapsed = datetime.now() - start_time
logger.info(f"⏱️ Thời gian chạy: {elapsed}")
logger.info("=== Hoàn tất điền cột 'year' (MusicBrainz + Wikipedia + iTunes) ===")

18:03:29 | INFO     | === Bắt đầu điền cột 'year' sử dụng MusicBrainz + Wikipedia + iTunes ===
18:03:31 | INFO     | Đã đọc 64983 hàng từ file: output/merged_yearFilled_final_2.csv
18:03:31 | INFO     | Cần điền 'year' cho 7099 bài hát
18:03:31 | INFO     | 🚀 Sử dụng 10 threads để tăng tốc...
Filling year:  30%|██▉       | 2112/7099 [10:12<24:07,  3.45it/s]
